we use convex Hall(Algoritm) it recognize all outer edge pixal and connect them of all object .

We firstly do it for images Firstly


In [8]:
import cv2
import numpy as np
hand=cv2.imread(r'C:\Users\Asus\Downloads\additional_stdy\opencv_tutorial\hand.jpg',0)#convert gray Scale and than read
#print(hand)
hand=cv2.resize(hand,(500,500))
ret,the=cv2.threshold(hand,127,270,cv2.THRESH_BINARY)#conversion in Binery Image
#ret,the=return,threshold
contours,_=cv2.findContours(the.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# it return two values we will use only contours  cv2.findContours(copy of thresholded image,type of cantor,Approximation function)
#here we try to get contours it help to get outer area and try to get connected pixal,use of Approxmation function we use it to change countour function
#print(contours)
#contours=max(contours,key=lambda x:cv2.contourArea(x))
hull=[cv2.convexHull(c) for c in contours]
#print(hull)
final=cv2.drawContours(hand,hull,-1,(250,270,0),0)
cv2.imshow("hand figure",hand)
cv2.imshow("thresholded image",the)
cv2.imshow("convex Hall images",final)
cv2.waitKey(0)
cv2.destroyAllWindows()

now we make final hand gesturese

In [16]:
import cv2
import numpy as np
import math
cap=cv2.VideoCapture(0)
while True:#while cap.isOpened(): both are valid
    ret,frame=cap.read()#capture frame from the Camera(opencv by default read image in  BGR formate)
    frame=cv2.flip(frame,8)#flip along y axis
    #cv2.rectangle(frame,(start coordinate x,y),(end coordinate of rectangle),(color B,G,R),width)
    cv2.rectangle(frame,(100,100),(300,300),(0,255,0),1)
    crop_image=frame[100:300,100:300]#cropped(taking image present in rectagle) frame[y:y+h, x:x+w]
    #Apply Gaussian blur(Smothing and filtering)
    blur=cv2.GaussianBlur(crop_image,(3,3),0)
    # conversion of BGR convert to HSV
    hsv=cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)#(opencv by default read image in  BGR format)
    #create a binery image with where white will be skin colors and rest s black
    mask2=cv2.inRange(hsv,np.array([0, 10, 60]),np.array([20, 150, 255]))
    #marphological function means any important information which lost  or worst valuewhich gain we removw that
    kernal=np.ones((5,5))#use kernal for that
    #Apply morphological Transfrmation to filter out the background noise
    erosion=cv2.erode(mask2,kernal,iterations=1)#apply erosion(remove unnessary data)
    dilation=cv2.dilate(erosion,kernal,iterations=1)#apply dilation(adding imporatant imformation which lost during inRange)
    
    #Apply Gaussian blur becuse it is possible that not all noise remove we filter that
    filtered=cv2.GaussianBlur(dilation,(3,3),0)
    ret,thre=cv2.threshold(filtered,127,255,0)#here we put 0 instead of cv2.THRESH_BINARY BOTH WORK SAMELY THERE
    
    cv2.imshow("Thresholded image",thre)
    #Find countours
    contours,hierarchy=cv2.findContours(thre.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    try:
        # Find contour with maximum area
        contour=max(contours,key=lambda x:cv2.contourArea(x))
        #Create bounding rectangle around the contour
        x,y,w,h=cv2.boundingRect(contour)
        cv2.rectangle(crop_image,(x,y),(x+w,y+h),(0,0,255),1)# here we draw rectangles
        # find convex hell
        
        hull=cv2.convexHull(contour)
        #draw contour
        drawing=np.zeros(crop_image.shape,np.uint8)#creating black images
        cv2.drawContours(drawing,[contour],-1,(0,255,0),0)#color of contor is green
        cv2.drawContours(drawing,[hull],-1,(0,0,255),0)#color of hull is Red
        #Find convexity defects for more details see this https://docs.opencv.org/3.0-beta/doc/...
        hull=cv2.convexHull(contour,returnPoints=False)
        defects=cv2.convexityDefects(contour,hull)
        
        #Use cosine rule to find angle of far point from the end point the  
        #for all defects
        count_defects=0
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
            
            # if angle > 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image, far, 1, [0, 0, 255], -1)

            cv2.line(crop_image, start, end, [0, 255, 0], 2)
        # Print number of fingers
        if count_defects == 0:
            cv2.putText(frame, "ONE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif count_defects == 1:
            cv2.putText(frame, "TWO", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 2:
            cv2.putText(frame, "THREE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 3:
            cv2.putText(frame, "FOUR", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif count_defects == 4:
            cv2.putText(frame, "FIVE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
            
    except:
        pass
    
    #Show all images
    cv2.imshow("Gesture",frame)# initial images with rectangle
    all_image=np.hstack((drawing,crop_image))
    cv2.imshow("Contours and hull images",all_image)
    
    #Closethe camera if 'q' is pressed
    if cv2.waitKey(1)==ord('q'):#so ord("key") if function which used to give assikey value of particulaer key
        break
    
    
cap.release()
cv2.destroyAllWindows()

In [11]:
cap.release()
cv2.destroyAllWindows()